<a href="https://colab.research.google.com/github/ChristophWippel/ird_app/blob/main/Another_copy_of_Ideal_response_dose_model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import ipywidgets as widgets
from ipywidgets import interactive_output, HBox, VBox, Layout, Button
from IPython.display import display

def create_plot(wl_target, initial_weight, height_m, lower_bmi, current_week, responder_type, data_version):
    study_weeks = 84

    # Convert parameters
    wl_target = float(wl_target)
    initial_weight = float(initial_weight)
    height_m = float(height_m)
    lower_bmi = float(lower_bmi)
    current_week = int(current_week)
    data_version = int(data_version)

    # Time axis for curves
    t = np.linspace(0, study_weeks, 100)

    # Ideal final weight based on WL target
    final_weight = initial_weight * (1 - wl_target/100)

    # Ideal weight loss curve (exponential decay)
    ideal_weight = initial_weight - (initial_weight - final_weight) * (1 - np.exp(-t / 30))

    # Lower limit weight loss curve (based on BMI)
    bmi_limit_weight = lower_bmi * (height_m ** 2)
    lower_limit_weight = initial_weight - (initial_weight - bmi_limit_weight) * (1 - np.exp(-t / 30))

    # Measurement points (every 4 weeks)
    weeks = np.arange(0, study_weeks + 1, 4)
    ideal_at_measurements = np.interp(weeks, t, ideal_weight)

    # Generate offsets based on responder type
    np.random.seed(42 + data_version)
    N = len(weeks)
    offsets = np.zeros(N)

    if responder_type == "High Responder":
        # Example piecewise: deeper in first half, moderate in second half
        for i in range(N):
            p = i/(N-1) if N>1 else 0
            if p < 0.5:
                target = -32 * p
            else:
                target = -16 + 4*(p - 0.5)
            offsets[i] = target + np.random.normal(0, 1.5)
    elif responder_type == "Well Responder":
        for i in range(N):
            p = i/(N-1) if N>1 else 0
            offsets[i] = -8*p + np.random.normal(0, 1)
    elif responder_type == "Low Responder":
        for i in range(N):
            p = i/(N-1) if N>1 else 0
            offsets[i] = (1 + 3*p) + np.random.normal(0, 1.5)

    actual_weight = ideal_at_measurements + offsets
    actual_weight[0] = initial_weight  # Ensure baseline is exact

    # Reveal data only up to current_week
    mask = weeks <= current_week
    revealed_weeks = weeks[mask]
    revealed_weights = actual_weight[mask]

    # Create the figure/axes
    fig, ax = plt.subplots(figsize=(8, 5))

    # Fix X-limits at 0..84 and Y-limits at 55..(initial_weight+5)
    ax.set_xlim(0, study_weeks)
    ax.set_ylim(55, initial_weight + 5)

    # Colors
    ideal_color = (63/255, 156/255, 53/255, 0.7)
    lower_limit_color = (1, 0, 0, 0.7)
    red_shaded_color = (1, 0, 0, 0.15)
    green_shaded_alpha = 0.2
    orange_shaded_color = (204/255, 197/255, 189/255, 0.3)
    patient_color = (0, 25/255, 101/255)

    # Top of the plot (fixed) for gray fill
    top_of_plot = ax.get_ylim()[1]

    # 1) Fill gray area from the top of the plot down to the ideal curve
    ax.fill_between(t, [top_of_plot]*len(t), ideal_weight, color=orange_shaded_color, zorder=0)

    # 2) Fill green area between ideal and lower-limit curves
    ax.fill_between(t, ideal_weight, lower_limit_weight, color=ideal_color, alpha=green_shaded_alpha, zorder=1)

    # 3) Fill red area from the lower-limit curve down to the bottom of the plot (y=55)
    ax.fill_between(t, lower_limit_weight, 55, color=red_shaded_color, zorder=2)

    # Plot ideal and lower-limit curves
    ax.plot(t, ideal_weight, color=ideal_color, linewidth=1.5, zorder=3)
    ax.plot(t, lower_limit_weight, color=lower_limit_color, linewidth=1.5, zorder=3)

    # Plot patient data with clip_on=False so the first & last markers aren't cut off
    if len(revealed_weeks) > 0:
        line, = ax.plot(revealed_weeks, revealed_weights, 'o-', color=patient_color,
                        linewidth=2, markersize=6, zorder=4, clip_on=False)
        line.set_clip_on(False)

    # If the full study is revealed, annotate the achieved % WL
    if current_week >= study_weeks:
        achieved_pct = 100 * (initial_weight - actual_weight[-1]) / initial_weight
        final_y_data = revealed_weights[-1]
        frac_y = (final_y_data - ax.get_ylim()[0]) / (ax.get_ylim()[1] - ax.get_ylim()[0])
        ax.text(1.01, frac_y, f"Achieved: {achieved_pct:.1f}%", transform=ax.transAxes,
                clip_on=False, color=patient_color, fontsize=10, fontweight='bold', va='center')

    # Place WL target and BMI lower annotations outside the plot
    final_ideal_y = ideal_weight[-1]
    final_lower_y = lower_limit_weight[-1]
    frac_ideal = (final_ideal_y - ax.get_ylim()[0]) / (ax.get_ylim()[1] - ax.get_ylim()[0])
    frac_lower = (final_lower_y - ax.get_ylim()[0]) / (ax.get_ylim()[1] - ax.get_ylim()[0])
    ax.text(1.01, frac_ideal, f"WL target: {wl_target}%", transform=ax.transAxes,
            clip_on=False, color=ideal_color, fontsize=10, va='center')
    ax.text(1.01, frac_lower, f"BMI lower: {lower_bmi}", transform=ax.transAxes,
            clip_on=False, color=lower_limit_color, fontsize=10, va='center')

    # Baseline BMI annotation at top center
    baseline_bmi = initial_weight / (height_m**2)
    fig.text(0.5, 0.95, f"Baseline BMI: {baseline_bmi:.2f}", ha='center', fontsize=12, fontweight='bold')

    ax.set_xlabel("Study Week")
    ax.set_ylabel("Body Weight (kg)")
    ax.set_xticks(np.arange(0, study_weeks+1, 4))
    ax.grid(True, linestyle='--', alpha=0.6)
    ax.set_title("Weight Loss Trajectory")

    plt.show()

# --- Interactive Controls (unchanged) ---
current_week_slider = widgets.IntSlider(value=0, min=0, max=84, step=4, description="Reveal Week")

prev_button = Button(description='← Previous', layout=Layout(width='120px'))
next_button = Button(description='Next →', layout=Layout(width='120px'))

def on_prev(b):
    current_week_slider.value = max(current_week_slider.min, current_week_slider.value - 4)

def on_next(b):
    current_week_slider.value = min(current_week_slider.max, current_week_slider.value + 4)

prev_button.on_click(on_prev)
next_button.on_click(on_next)

responder_type_dropdown = widgets.Dropdown(
    options=["High Responder", "Well Responder", "Low Responder"],
    value="Well Responder",
    description="Responder Type"
)

wl_target_slider = widgets.IntSlider(value=25, min=0, max=50, step=5, description="WL Target (%)")
initial_weight_slider = widgets.FloatSlider(value=95, min=50, max=200, step=5, description="Initial Weight (kg)")
height_slider = widgets.FloatSlider(value=1.65, min=1.4, max=2.0, step=0.01, description="Height (m)")
lower_bmi_slider = widgets.FloatSlider(value=20, min=15, max=30, step=1, description="Lower BMI Limit")

data_version_slider = widgets.IntSlider(value=0, min=0, max=1000, step=1, description="Data Version")
data_version_slider.layout.display = 'none'

generate_button = Button(description="Generate New Data", button_style='info')
def on_generate(b):
    data_version_slider.value += 1
generate_button.on_click(on_generate)

button_box = HBox([prev_button, next_button, current_week_slider])
controls = VBox([
    wl_target_slider,
    initial_weight_slider,
    height_slider,
    lower_bmi_slider,
    responder_type_dropdown,
    generate_button,
    button_box,
    data_version_slider
])

out = interactive_output(create_plot, {
    'wl_target': wl_target_slider,
    'initial_weight': initial_weight_slider,
    'height_m': height_slider,
    'lower_bmi': lower_bmi_slider,
    'current_week': current_week_slider,
    'responder_type': responder_type_dropdown,
    'data_version': data_version_slider
})

display(controls, out)


Output()